In [43]:
### Illustration du biais de regularisation
### 26 septembre 2019
### Jeremy LHour

library("MASS")
library("glmnet")
library("ggplot2")
library("gridExtra")

### fonctions utilisateur
source("functions/DataSim.R") 

# Post Lasso
PostLasso <- function(X,y,nopen){
    #'@param nopen indices des variables a ne pas penaliser
    phi = rep(1,ncol(X)); phi[nopen] = 0
    # ETAPE 1
    cv.lasso = cv.glmnet(X,y, family="gaussian",alpha=1,penalty.factor=phi,nfolds=5)
    b.lasso = coef(cv.lasso); b.lasso = b.lasso[-1] # on enleve la constante
    s.hat = which(b.lasso != 0) # indices des variables actives parmi les X
    # ETAPE 2
    postlasso = lm(y ~ X[,s.hat])
    b.pl = coef(postlasso)

    return(b.pl)
}

# Post Student
PostStudent <- function(X,y,noppen,alpha=.05){
    #'@param nopen indices des variables a ne pas penaliser
    #'@param alpha niveau de significativite pour selection
    # ETAPE 1
    fullreg = lm(y ~ X)
    s.hat = summary(fullreg)$coef[,"Pr(>|t|)"] < alpha
    s.hat = s.hat[-1]
    s.hat = union(s.hat,nopen)
    # ETAPE 2
    shortreg = lm(y ~ X[,s.hat])
    b.ps = coef(shortreg)

    return(b.ps)
}

In [44]:
### Simulations
R = 1000 # nb simulations
n = 2000 # sample size
p = 10 # nb variables
tau = 0 # vrai valeur de l'effet

set.seed(326)
Results = matrix(ncol=3, nrow=R)

In [45]:
t_start = Sys.time()

for(r in 1:R){
  ### GENERATE DATA
  data = suppressWarnings(DataSim(n=n,p=p,Ry=.3,Rd=.7,Intercept=F,a=tau))
  X = cbind(data$d,data$X); y = data$y
  
  ### METHOD 1: Modele complet
  full.reg = lm(y ~ X)

  ### METHOD 2: Selection Lasso
  lasso.reg = PostLasso(X,y,nopen=c(1))
  
  ### METHOD 3: Selection sur la stat de Student
  student.reg = PostStudent(X,y,noppen=c(1),alpha=.05)
  
  
  ### COLLECTING RESULTS
  Results[r,] = c(full.reg$coef[2],
                  lasso.reg[2],
                  student.reg[2])
}

print(Sys.time()-t_start)

Time difference of 1.855768 mins


In [46]:
### COMPUTE BIAS AND RMSE
StatDisplay = data.frame()
StatDisplay[1:3,"bias"] = apply(Results-tau,2,mean)
StatDisplay[1:3,"RMSE"] = sqrt(apply((Results-tau)^2,2,mean))
row.names(StatDisplay) = c("Complet","Post-Lasso","Post-Student")
print(StatDisplay)

                     bias       RMSE
Complet      -0.001452197 0.05969442
Post-Lasso    0.025022958 0.06241013
Post-Student  1.090680768 1.09209092


In [47]:
### DRAW CHARTS
id = c(mapply(function(x) rep(x,R),1:3))
val = c(Results)-tau
data_res = data.frame(val = val, model = id)

M = max(abs(quantile(Results,.01,na.rm=T)),abs(quantile(Results,.99,na.rm=T)))
lb = -1.1*M; ub = 1.1*M

get.plot <- function(data,modelS,title="A Title",s){
  plot_res <- ggplot(subset(data, (model==modelS)), aes(x=val)) + 
    geom_histogram(binwidth = .02, alpha=.5, position='identity',fill="steelblue", aes(y = ..density..)) +
    scale_x_continuous(limits=c(lb,ub), name="Treatment effect") +
    ggtitle(title) + 
    stat_function(fun = dnorm, args=list(mean=0, sd=s), colour="darkorchid3", size=1) +
    theme(plot.title = element_text(lineheight=.8, face="bold"),legend.position="none")
  return(plot_res)
} # plot func


grid.arrange(get.plot(data_res,1,"Modele Complet", StatDisplay[1,2]), get.plot(data_res,2,"Post-Lasso",  StatDisplay[1,2]), get.plot(data_res,3,"Double Selection",  StatDisplay[1,2]), ncol=3)


ERROR: Error in grid.arrange(get.plot(data_res, 1, "Modele Complet", StatDisplay[1, : could not find function "grid.arrange"
